# Result Arrangement

In [1]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re


import numpy as np
import matplotlib.pyplot as plt

In [2]:
import Basic_Module as bm

## Category 

In [3]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## Path

In [4]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'
outpath = './outcome_news_classification/'

In [7]:
outcomeFileList = glob(outpath+'outcome_news_classification'+'*')
print (len(outcomeFileList))

18


In [6]:
outDf = pd.DataFrame()
for outcomeFile in outcomeFileList:
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    df.set_index(keys='id', inplace=True)
    modelIs = bm.ExtractModelType(outcomeFile)
    print (modelIs)

Doc2Vec_dbow
fastText_CBOW_cbow_mean-1
Doc2Vec_dm-c
Doc2Vec_dm-c
Doc2Vec_dbow
Doc2Vec_dm-m
word2vec_CBOW_cbow_mean-0
fastText_CBOW_cbow_mean-0
word2vec_CBOW_cbow_mean-1
fastText_CBOW_cbow_mean-1
word2vec_CBOW_cbow_mean-1
word2vec_CBOW_cbow_mean-0
fastText_skip-gram_cbow_mean-0
Doc2Vec_dm-m
word2vec_skip-gram_cbow_mean-0
fastText_skip-gram_cbow_mean-0
fastText_CBOW_cbow_mean-0
word2vec_skip-gram_cbow_mean-0


In [ ]:
pd.re